## Launch an Inference Server (vLLM) for the Compressed Model

This step sets up a vLLM inference server to host your compressed model and exposes an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance like throughput, latency, and time-to-first-token.

**Goal**: Make the compressed model accessible via an API for performance evaluation.

**Output**: vLLM server running with the compressed model, ready to accept requests.


In [1]:
import os
import time
import torch
from utils import generate

/opt/app-root/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

### vLLM config for single node
For **single-node**, **single-GPU** or **multi-GPU (but not multinode)** vLLM serving, the main arguments are:

``--model``: Model path or Hugging Face repo ID (required).

``--tensor-parallel-size``: Number of GPUs to use (set to 1 for single GPU, or >1 for multi-GPU tensor parallelism).

``--port``: Port for the API server (default is 8000).

``--host``: Host IP address (default is 127.0.0.1).

``--gpu-memory-utilization``: controls what fraction of each GPU’s memory vLLM will use for the model executor and KV cache. For example, --gpu-memory-utilization 0.5 means vLLM will use 50% of the GPU memory.

``--quantization``: Method used to quantize the weights. 
 
``--max-model-len``: argument sets the maximum context length (in tokens) that vLLM can process for both prompt and output combined. If not specified, it defaults to the model’s config value. Setting a higher value allows longer prompts/completions but increases GPU memory usage for the KV cache; setting it lower saves memory but limits context length. Set this to prevent problems with memory if the model’s default context length is too long.

For **multi-node** vLLM serving, use:

 ``--tensor-parallel-size`` Number of GPUs per node (or total GPUs if not using pipeline parallelism).
 
``--pipeline-parallel-size`` Number of nodes (optional, for pipeline parallelism).

Additionally, for multi-node setup, a Ray cluster is also needed.



# Run this command in terminal to serve the compressed model using vLLM
```
vllm serve \
  --model "compressed_model" \
  --host 127.0.0.1 \
  --port 8000 \
  --gpu-memory-utilization 0.8 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1
```

Once the server starts, you will see something like this:

```INFO:     Started server process [37883]```\
```INFO:     Waiting for application startup.```\
```INFO:     Application startup complete.```


## A test run to see if the vLLM server is accessible
We use a helper function **generate** (defined in [utils.py](./utils.py)) to simplify sending requests to our locally-served VLLM model.
This function wraps the OpenAI-compatible Chat Completions API exposed by VLLM.
### Why we Use the OpenAI SDK with vLLM
vLLM implements an OpenAI-compatible REST API, meaning:

- vLLM starts a local web server (e.g., http://127.0.0.1:8000/v1)

- it exposes the same endpoints (/v1/chat/completions, /v1/completions) as OpenAI

- accepts the same request schema (messages=[{"role": "..."}])

- the same client interface as OpenAI

- The OpenAI SDK doesn't know (or care) that it isn’t talking to OpenAI — it just sends HTTP requests to the specified url in the expected format

An **alternate** way is to send POST requests using python's **requests** module.


In [4]:
response = generate("compressed_model", "Explain quantum computing simply?")
print(response)

Quantum computing is a complex topic, but I'll try to break it down in simple terms.

**Classical Computing**

Imagine you have a combination lock with 10 numbers (0-9). To open the lock, you need to try each number one by one, until you find the correct combination. This is like a classical computer, which uses "bits" (0s and 1s) to process information.

**Quantum Computing**

Now, imagine you have a special lock that can try all 10 numbers simultaneously, and it will open the lock as soon as it finds the correct combination. This is like a quantum computer, which uses "qubits" (quantum bits) to process information.

Qubits are special because they can exist in multiple states at the same time, unlike classical bits which are either 0 or 1. This property, called superposition, allows quantum computers to process multiple possibilities simultaneously, making them incredibly fast for certain types of calculations.

**How it Works**

Quantum computers use quantum bits (qubits) to perform